# Generate Orders from Positions

In [1]:
import io
import datetime
import pytz

import pandas as pd
import numpy as np

## Data

In [2]:
UTC_TZ = pytz.timezone("UTC")

def read(s):
    df = pd.read_csv(io.StringIO(s), sep="|")
    df.rename(columns=lambda s: s.strip(), inplace=True)
    for col in df.columns:
        if col in ["dt", "rt"]:
            df[col] = pd.to_datetime(df[col])
        elif df.dtypes[col] == np.dtype("O"):
            df[col] = df[col].apply(lambda s: s.strip())
        else:
            pass
    return df.set_index([c for c in df.columns if c != "value"])['value']

1. rt - reference time: when the strategy wants to achieve given target position
2. dt - decision time: when the strategy decided to achieve that position

4ex: Strategy S1 runs at 9am and generates a target position of 10e6 EUR at 9:30am, `dt` will be 9am and `rt` will be 9:30.

In [3]:
# OG Target Positions

# tpos = read("""
#     rt                   | strategy | asset | dt                   | value
#     2023-08-29 07:00:00z | s1       | PLN   | 2023-08-29 07:00:00z | 4e6
#     2023-08-29 16:00:00z | s1       | CZK   | 2023-08-29 07:00:00z | -24e6
#     2023-08-30 07:00:00z | s1       | PLN   | 2023-08-30 07:00:00z | 8e6
#     2023-08-30 16:00:00z | s1       | CZK   | 2023-08-30 07:00:00z | -48e6
#     2023-08-29 07:00:00z | s2       | EUR   | 2023-08-29 07:00:00z | 1.8e6
#     2023-08-29 16:00:00z | s2       | AUD   | 2023-08-29 07:00:00z | -1.5e6
#     2023-08-30 07:00:00z | s2       | EUR   | 2023-08-30 07:00:00z | 3.6e6
#     2023-08-30 16:00:00z | s2       | AUD   | 2023-08-30 07:00:00z | -1e6
#     2023-08-29 07:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 12e6
#     2023-08-29 16:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 18e6
#     2023-08-30 07:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 18e6
#     2023-08-30 16:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 24e6
#     """)

In [4]:
# Tested Target Positions

tpos = read("""
    rt                   | strategy | asset | dt                   | value
    2023-08-29 07:00:00z | s1       | PLN   | 2023-08-29 07:00:00z | 4e6
    2023-08-29 16:00:00z | s1       | CZK   | 2023-08-29 07:00:00z | -24e6
    2023-08-30 07:00:00z | s1       | PLN   | 2023-08-30 07:00:00z | 8e6
    2023-08-30 16:00:00z | s1       | CZK   | 2023-08-30 07:00:00z | -48e6
    2023-08-29 07:00:00z | s2       | EUR   | 2023-08-29 07:00:00z | 1.8e6
    2023-08-29 16:00:00z | s2       | AUD   | 2023-08-29 07:00:00z | -1.5e6
    2023-08-30 07:00:00z | s2       | EUR   | 2023-08-30 07:00:00z | 3.6e6
    2023-08-30 16:00:00z | s2       | AUD   | 2023-08-30 07:00:00z | -1e6
    2023-08-29 07:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 12e6
    2023-08-29 16:00:00z | s3       | CZK   | 2023-08-29 07:00:00z | 18e6
    2023-08-29 16:00:00z | s3       | PLN   | 2023-08-29 07:00:00z | 10e6
    2023-08-30 07:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 18e6
    2023-08-30 16:00:00z | s1       | PLN   | 2023-08-30 07:00:00z | 9.2e6
    2023-08-30 16:00:00z | s3       | PLN   | 2023-08-30 07:00:00z | 10.2e6
    2023-08-30 16:00:00z | s3       | CZK   | 2023-08-30 07:00:00z | 24e6
    """)

In [5]:
#Expected fx rates at different times?
fx_rates = read("""
    rt                   | asset | value
    2023-08-29 07:00:00z | PLN   | 3.934
    2023-08-29 16:00:00z | PLN   | 3.924
    2023-08-30 07:00:00z | PLN   | 3.914
    2023-08-30 16:00:00z | PLN   | 3.904
    
    2023-08-29 07:00:00z | CZK   | 23.12
    2023-08-29 16:00:00z | CZK   | 23.08
    2023-08-30 07:00:00z | CZK   | 23.02
    2023-08-30 16:00:00z | CZK   | 23.01
    
    2023-08-29 07:00:00z | EUR   | 1.116
    2023-08-29 16:00:00z | EUR   | 1.119
    2023-08-30 07:00:00z | EUR   | 1.121
    2023-08-30 16:00:00z | EUR   | 1.122
    
    2023-08-29 07:00:00z | AUD   | 0.672
    2023-08-29 16:00:00z | AUD   | 0.682
    2023-08-30 07:00:00z | AUD   | 0.689
    2023-08-30 16:00:00z | AUD   | 0.690
    """)

In [6]:
#For exercise 3, minimum order size
min_order_size_usd = read("""
    asset | value
    PLN   | 5e5
    CZK   | 5e5
    EUR   | 1e6
    AUD   | 1e6
""")

In [7]:
#Date at which the 
trading_session = pd.Timestamp("2023-08-30 16:00:00")

# Easy Task: Generate Orders (total and by strategy) as of 2023-08-30 16:00 (Local Ccy)

In [8]:
df_order_local_currency = tpos.unstack(['strategy','asset']).ffill().diff().xs(trading_session,level='rt').iloc[0].rename('order_local_ccy').reset_index()

In [9]:
df_order_local_currency

,strategy,asset,order_local_ccy
0,s1,PLN,1200000.0
1,s1,CZK,-24000000.0
2,s2,EUR,0.0
3,s2,AUD,500000.0
4,s3,CZK,6000000.0
5,s3,PLN,200000.0


In [10]:
df_order_local_currency.groupby('asset')['order_local_ccy'].sum().reset_index()

,asset,order_local_ccy
0,AUD,500000.0
1,CZK,-18000000.0
2,EUR,0.0
3,PLN,1400000.0


In [11]:
order_by_strategy_and_asset_local_ccy = pd.DataFrame([
    {'strategy': 's1', 'asset': 'PLN', 'order_local_ccy': 0.0},
    {'strategy': 's1', 'asset': 'CZK', 'order_local_ccy': -24000000.0},
    {'strategy': 's2', 'asset': 'EUR', 'order_local_ccy': 0.0},
    {'strategy': 's2', 'asset': 'AUD', 'order_local_ccy': 500000.0},
    {'strategy': 's3', 'asset': 'CZK', 'order_local_ccy': 6000000.0}
])
order_by_strategy_and_asset_local_ccy

,strategy,asset,order_local_ccy
0,s1,PLN,0.0
1,s1,CZK,-24000000.0
2,s2,EUR,0.0
3,s2,AUD,500000.0
4,s3,CZK,6000000.0


In [12]:
order_by_asset_local_ccy = pd.DataFrame([
    {'asset': 'AUD', 'order_local_ccy': 500000.0},
    {'asset': 'CZK', 'order_local_ccy': -18000000.0},
    {'asset': 'EUR', 'order_local_ccy': 0.0},
    {'asset': 'PLN', 'order_local_ccy': 0.0}
])
order_by_asset_local_ccy

,asset,order_local_ccy
0,AUD,500000.0
1,CZK,-18000000.0
2,EUR,0.0
3,PLN,0.0


# Easy / Medium Task: Generate Orders (total and by strategy) as of 2023-08-30 16:00 (USD)

*Note inverted prices of EUR and AUD*

In [13]:
fx_rates_selected = fx_rates.xs(trading_session,level = 'rt')

In [14]:
df_order_with_usd = df_order_local_currency.copy()

In [15]:
df_order_with_usd['fx_rate'] = df_order_local_currency['asset'].apply(lambda x : 1/fx_rates_selected[x] if x in ['EUR','AUD'] else fx_rates_selected[x])
df_order_with_usd['order_usd'] = df_order_with_usd['order_local_ccy'] / df_order_with_usd['fx_rate']

In [16]:
df_order_with_usd.drop(['order_local_ccy','fx_rate'],axis=1)

,strategy,asset,order_usd
0,s1,PLN,3.073770e+05
1,s1,CZK,-1.043025e+06
2,s2,EUR,0.000000e+00
3,s2,AUD,3.450000e+05
4,s3,CZK,2.607562e+05
5,s3,PLN,5.122951e+04


In [17]:
orders_by_asset_usd = df_order_with_usd.groupby('asset')['order_usd'].sum().reset_index()
orders_by_asset_usd

,asset,order_usd
0,AUD,345000.000000
1,CZK,-782268.578879
2,EUR,0.000000
3,PLN,358606.557377


In [18]:
order_by_strategy_and_asset_usd = pd.DataFrame([
    {'strategy': 's1', 'asset': 'PLN', 'order_usd': 0.0},
    {'strategy': 's1', 'asset': 'CZK', 'order_usd': -1043024.7718383311},
    {'strategy': 's2', 'asset': 'EUR', 'order_usd': 0.0},
    {'strategy': 's2', 'asset': 'AUD', 'order_usd': 344999.99999999994},
    {'strategy': 's3', 'asset': 'CZK', 'order_usd': 260756.19295958278}
])
order_by_strategy_and_asset_usd

,strategy,asset,order_usd
0,s1,PLN,0.000000e+00
1,s1,CZK,-1.043025e+06
2,s2,EUR,0.000000e+00
3,s2,AUD,3.450000e+05
4,s3,CZK,2.607562e+05


In [19]:
order_by_asset_usd = pd.DataFrame([
    {'asset': 'AUD', 'order_usd': 344999.99999999994},
    {'asset': 'CZK', 'order_usd': -782268.5788787483},
    {'asset': 'EUR', 'order_usd': 0.0},
    {'asset': 'PLN', 'order_usd': 0.0}
])
order_by_asset_usd

,asset,order_usd
0,AUD,345000.000000
1,CZK,-782268.578879
2,EUR,0.000000
3,PLN,0.000000


# Hard Task: Apply Minimum Order Size and generate new target position after this session

*Context: In practice there is often minimum cost we need to pay when trading. This means the orders must be of certain size to make economic sense. Therefore, if order is below the limit size it will not be executed and this needs to be fed back to the target position, so that during next strategy it trades orders knowing that past position is as of T-2, not T-1 (since we skipped T-1 orders)*

# My approach

## Note: This approach doesn't work if there's a position with empty values, e.g. Adding s3 PLN with just rt 08-30 16:00

In [20]:
#Get initial positions status
positions_initial = tpos.unstack(['strategy','asset']).ffill()

#Create an output df for clean position
positions_clean = positions_initial.copy()

#Get positions on a given rt date
positions_filtered = positions_initial.loc[trading_session]

#Copy valid orders df
valid_orders_usd = orders_by_asset_usd.copy()

#Verify whether a specific currency trade is valid. That is only if its absolute value is above the threshold or if it's 0
valid_orders_usd['validity'] = valid_orders_usd.apply(lambda row : (abs(row['order_usd']) >= min_order_size_usd[row['asset']]) | (row['order_usd']==0),axis=1)

#Identify which currencies are not valid and had any non zero order
impacted_assets = valid_orders_usd[valid_orders_usd['validity']==False]['asset']

#Find which strategies are impacted by invalid orders
df_order_local_currency[df_order_local_currency['asset'].isin(impacted_assets)]

#Create adjusted order
assets_strategies_to_adjust = df_order_local_currency[df_order_local_currency['asset'].isin(impacted_assets)]



#Since the validity threshold is on asset value granularity, we can assume that if the order doesnt pass, 
#every strategy trading that asset should be reverted
#hence we iterate through every strategy+asset combination in the initial order and then in our position table 
#we subtract those values from the desired position


#Iterate over each row in the adjustment DataFrame
for index, row in assets_strategies_to_adjust.iterrows():
    strategy = row['strategy']  # Get the strategy
    asset = row['asset']        # Get the asset
    adjustment_value = row['order_local_ccy']  # Amount to subtract
    
    #Locate the correct position in the 'positions_filtered' DataFrame and apply the adjustment
    if (strategy, asset) in positions_filtered.columns:
        print(f'Adjusting: {(positions_clean.loc[(trading_session), (strategy, asset)]).name} on {trading_session} by {adjustment_value}')
        # Subtract the adjustment value from the corresponding strategy-asset cell
        positions_clean.loc[(trading_session), (strategy, asset)][0] -= adjustment_value
       

positions_clean

Adjusting: ('s1', 'PLN') on 2023-08-30 16:00:00 by 1200000.0
Adjusting: ('s2', 'AUD') on 2023-08-30 16:00:00 by 500000.0
Adjusting: ('s3', 'PLN') on 2023-08-30 16:00:00 by 200000.0


C:\Users\jakub\AppData\Local\Temp\ipykernel_15752\113210771.py:43: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  positions_clean.loc[(trading_session), (strategy, asset)][0] -= adjustment_value
C:\Users\jakub\AppData\Local\Temp\ipykernel_15752\113210771.py:43: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] =

strategy                                        s1                     s2  \
asset                                          PLN         CZK        EUR   
rt                  dt                                                      
2023-08-29 07:00:00 2023-08-29 07:00:00  4000000.0         NaN  1800000.0   
2023-08-29 16:00:00 2023-08-29 07:00:00  4000000.0 -24000000.0  1800000.0   
2023-08-30 07:00:00 2023-08-30 07:00:00  8000000.0 -24000000.0  3600000.0   
2023-08-30 16:00:00 2023-08-30 07:00:00  8000000.0 -48000000.0  3600000.0   

strategy                                                    s3              
asset                                          AUD         CZK         PLN  
rt                  dt                                                      
2023-08-29 07:00:00 2023-08-29 07:00:00        NaN  12000000.0         NaN  
2023-08-29 16:00:00 2023-08-29 07:00:00 -1500000.0  18000000.0  10000000.0  
2023-08-30 07:00:00 2023-08-30 07:00:00 -1500000.0  18000000.0  10000000.0  
2023-08-30 16:00:00 2023-08-30 07:00:00 -1500000.0  24000000.0  10000000.0

# OG Solution

In [21]:
""" Minimum absolute size of the order in USD, below which we do not trade """
min_order_size_usd

asset
PLN     500000.0
CZK     500000.0
EUR    1000000.0
AUD    1000000.0
Name: value, dtype: float64

In [22]:
valid_orders_usd = pd.DataFrame([
    {'asset': 'AUD', 'order_usd': 344999.99999999994, 'valid_orders': 0.0},
    {'asset': 'CZK','order_usd': -782268.5788787483, 'valid_orders': -782268.5788787483},
    {'asset': 'EUR', 'order_usd': 0.0, 'valid_orders': 0.0},
    {'asset': 'PLN', 'order_usd': 0.0, 'valid_orders': 0.0}
])
valid_orders_usd

,asset,order_usd,valid_orders
0,AUD,345000.000000,0.000000
1,CZK,-782268.578879,-782268.578879
2,EUR,0.000000,0.000000
3,PLN,0.000000,0.000000


In [23]:
""" 
Here I find exactly which orders were scaled down and by what scalar, 
which I will need when calculating final orders by strategy and asset
"""
valid_orders_usd['fx_rate'] = valid_orders_usd['asset'].apply(
    lambda x: 1/fx_rates.loc[trading_session][x] if x in ['EUR', 'AUD'] else fx_rates.loc[trading_session][x]
)

valid_orders_usd['valid_orders_local'] = valid_orders_usd['valid_orders'] * valid_orders_usd['fx_rate']
valid_orders_usd['scalar'] = (valid_orders_usd['valid_orders'] / valid_orders_usd['order_usd']).fillna(0)
valid_orders_scalar = valid_orders_usd.set_index('asset')['scalar']
valid_orders_scalar

asset
AUD    0.0
CZK    1.0
EUR    0.0
PLN    0.0
Name: scalar, dtype: float64

In [24]:
o_sa_local = order_by_strategy_and_asset_local_ccy # for convenience
o_sa_local['order_local_ccy_validated'] = o_sa_local.apply(
    lambda row: valid_orders_scalar[row['asset']] * row['order_local_ccy'], axis=1
)
order_as_valid = o_sa_local.copy()
order_as_valid['dt'] = pd.Timestamp("2023-08-30 07:00:00z")
order_as_valid['rt'] = trading_session
order_as_valid

,strategy,asset,order_local_ccy,order_local_ccy_validated,dt,rt
0,s1,PLN,0.0,0.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00
1,s1,CZK,-24000000.0,-24000000.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00
2,s2,EUR,0.0,0.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00
3,s2,AUD,500000.0,0.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00
4,s3,CZK,6000000.0,6000000.0,2023-08-30 07:00:00+00:00,2023-08-30 16:00:00


In [25]:
""" Adding those orders back to positions to create final target position """

original_positions = tpos.unstack(['strategy', 'asset']).ffill()
original_positions_drop_last_row = original_positions[
    original_positions.index.get_level_values('rt') != trading_session]

clean_new_orders = order_as_valid.set_index(['rt', 'dt', 'strategy', 'asset'])['order_local_ccy_validated']

new_last_row = original_positions_drop_last_row.iloc[-1] + clean_new_orders

In [26]:
original_positions_drop_last_row.iloc[-1]

strategy  asset
s1        PLN       8000000.0
          CZK     -24000000.0
s2        EUR       3600000.0
          AUD      -1500000.0
s3        CZK      18000000.0
          PLN      10000000.0
Name: (2023-08-30 07:00:00, 2023-08-30 07:00:00), dtype: float64

In [27]:
""" Final Result """
modified_position_final = pd.concat([original_positions_drop_last_row, new_last_row.unstack(['strategy', 'asset'])], axis=0)
modified_position_final

strategy                                              s1              \
asset                                                PLN         CZK   
rt                  dt                                                 
2023-08-29 07:00:00 2023-08-29 07:00:00        4000000.0         NaN   
2023-08-29 16:00:00 2023-08-29 07:00:00        4000000.0 -24000000.0   
2023-08-30 07:00:00 2023-08-30 07:00:00        8000000.0 -24000000.0   
NaT                 NaN                              NaN         NaN   
2023-08-30 16:00:00 2023-08-30 07:00:00+00:00  8000000.0 -48000000.0   

strategy                                              s2             \
asset                                                EUR        AUD   
rt                  dt                                                
2023-08-29 07:00:00 2023-08-29 07:00:00        1800000.0        NaN   
2023-08-29 16:00:00 2023-08-29 07:00:00        1800000.0 -1500000.0   
2023-08-30 07:00:00 2023-08-30 07:00:00        3600000.0 -1500000.0   
NaT                 NaN                              NaN        NaN   
2023-08-30 16:00:00 2023-08-30 07:00:00+00:00  3600000.0 -1500000.0   

strategy                                               s3              
asset                                                 CZK         PLN  
rt                  dt                                                 
2023-08-29 07:00:00 2023-08-29 07:00:00        12000000.0         NaN  
2023-08-29 16:00:00 2023-08-29 07:00:00        18000000.0  10000000.0  
2023-08-30 07:00:00 2023-08-30 07:00:00        18000000.0  10000000.0  
NaT                 NaN                               NaN         NaN  
2023-08-30 16:00:00 2023-08-30 07:00:00+00:00  24000000.0         NaN